<a href="https://colab.research.google.com/github/Inkyu-Yang356/deep-learning-practice/blob/main/250521_text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 17장 딥러닝을 이용한 자연어 처리

## 1. 텍스트의 토큰화

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer  # 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences  # 패딩
from tensorflow.keras.models import Sequential  # 모델 구성
from tensorflow.keras.layers import Dense,Flatten,Embedding  # 층 구성
from tensorflow.keras.utils import to_categorical  # 카테고리화 (One-Hot-Encoding)
from numpy import array
from tensorflow.keras.preprocessing.text import text_to_word_sequence  # 토큰화


text = '해보지 않으면 해낼 수 없다'

# 토큰화
result = text_to_word_sequence(text)  # 빈 칸을 기준으로 토큰화
print("\n원문:\n", text)
print("\n토큰화:\n", result)


원문:
 해보지 않으면 해낼 수 없다

토큰화:
 ['해보지', '않으면', '해낼', '수', '없다']


### 단어 빈도 수 세기

In [2]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
       '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
       '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.',
       ]

# Tokenizer를 통한 전처리 준비 및 토큰화 함수에 문장 적용
token = Tokenizer()
token.fit_on_texts(docs)  # 전처리

print("\n단어 카운트:\n", token.word_counts) # 순서 O, 단어 빈도수

print("\n문장 카운트: ", token.document_count)
print("\n각 단어가 몇 개의 문장에 포함되어 있는가:\n", token.word_docs) # 순서 X
print("\n각 단어에 매겨진 인덱스 값:\n",  token.word_index) # 순서 X


단어 카운트:
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])

문장 카운트:  3

각 단어가 몇 개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'토큰화합니다': 1, '먼저': 1, '단어를': 1, '각': 1, '나누어': 1, '텍스트의': 2, '딥러닝에서': 2, '단어로': 1, '토큰화해야': 1, '인식됩니다': 1, '토큰화한': 1, '사용할': 1, '있습니다': 1, '수': 1, '결과는': 1})

각 단어에 매겨진 인덱스 값:
 {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15}


## 2. 단어의 원-핫 인코딩

In [3]:
text="오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"
token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [4]:
x=token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [5]:
# 인덱스 수에 하나를 추가해서 원-핫 인코딩 배열 만들기
word_size = len(token.word_index) + 1
x = to_categorical(x, num_classes=word_size)  # 범주형 인코딩
print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


In [15]:
token.word_index

{'너무': 1,
 '재밌네요': 2,
 '최고예요': 3,
 '참': 4,
 '잘': 5,
 '만든': 6,
 '영화예요': 7,
 '추천하고': 8,
 '싶은': 9,
 '영화입니다': 10,
 '한번': 11,
 '더': 12,
 '보고싶네요': 13,
 '글쎄요': 14,
 '별로예요': 15,
 '생각보다': 16,
 '지루하네요': 17,
 '연기가': 18,
 '어색해요': 19,
 '재미없어요': 20}

## 4.텍스트를 읽고 긍정, 부정 예측하기

In [6]:
# 영화 리뷰 데이터
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스 정의
classes = array([1,1,1,1,1,0,0,0,0,0])

# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index) # 각 단어별 index 매겨 출력


{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [7]:
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n",  x)


리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [8]:
# 서로 다른 길이의 데이터를 4로 맞추기 (패딩)
padded_x = pad_sequences(x, 4)
print("\n패딩 결과:\n", padded_x)  # 오른쪽 정렬


패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [9]:
# 임베딩 단어 수 지정
word_size = len(token.word_index) +1  # 20+1

# 워드 임베팅을 포함한 모델 생성
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))  # (21, 8, 4)
model.add(Flatten())  # 1차원 벡터화
model.add(Dense(1, activation='sigmoid'))  # FC Layer
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))  # 모델 평가

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.6921
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.5000 - loss: 0.6901
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.5000 - loss: 0.6882
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.6000 - loss: 0.6863
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.6000 - loss: 0.6844
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.7000 - loss: 0.6825
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.7000 - loss: 0.6806
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.9000 - loss: 0.6786
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.9000 - loss: 0.6767
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.9000 - loss: 0.6748
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 1.0000 - loss: 0.6729
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 1.0000 - loss: 

In [11]:
predictions = model.predict(padded_x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step


In [13]:
predictions

array([[0.5111511 ],
       [0.50088465],
       [0.52543646],
       [0.5294549 ],
       [0.537705  ],
       [0.4877131 ],
       [0.47933495],
       [0.48100263],
       [0.4634933 ],
       [0.49054548]], dtype=float32)

확인문제 1 : 주어진 리뷰를 기준으로 긍정/부정을 예측했을 때 예측한 분류값을 출력하는 예시입니다. 빈칸을 채워보세요.

In [22]:
for text, pred in zip(docs, predictions):
    sentiment = "긍정" if pred > 0.5 else "부정"
    print(f"Text: {text}\n예측 결과: {sentiment} ({pred[0]:.2f})\n")

print(predictions,'\n')

Text: 너무 재밌네요
예측 결과: 긍정 (0.51)

Text: 최고예요
예측 결과: 긍정 (0.50)

Text: 참 잘 만든 영화예요
예측 결과: 긍정 (0.53)

Text: 추천하고 싶은 영화입니다
예측 결과: 긍정 (0.53)

Text: 한번 더 보고싶네요
예측 결과: 긍정 (0.54)

Text: 글쎄요
예측 결과: 부정 (0.49)

Text: 별로예요
예측 결과: 부정 (0.48)

Text: 생각보다 지루하네요
예측 결과: 부정 (0.48)

Text: 연기가 어색해요
예측 결과: 부정 (0.46)

Text: 재미없어요
예측 결과: 부정 (0.49)

[[0.5111511 ]
 [0.50088465]
 [0.52543646]
 [0.5294549 ]
 [0.537705  ]
 [0.4877131 ]
 [0.47933495]
 [0.48100263]
 [0.4634933 ]
 [0.49054548]] 



확인 문제2 : 위와 같은 정확도와 예측 결과가 나타나는 이유를 적어보세요

- 위 문제는 이진 분류이며 출력 레이어 활성화 함수가 Sigmoid로 출력값이 0과 1사이입니다.
- 따라서, 중간값인 0.5를 기준으로 긍정과 부정으로 나뉩니다.
- 또, 데이터가 복잡하지 않지않아 쉽게 학습되고, 데이터 크기가 작기 때문에 과적합되었을 가능성이 있습니다.